# Import basic libraries

Let's import some basic stuff

## TODO
- Add the nodes in the CityJSON file, keeping the relationships between nodes and edges (linestrings). The nodes have semantics and atttributes.
- Find a small area and compute widths.
- Create LoD0.1 with single lines and attributes for two-way and same for LoD0.2.

# Ideas for modelling

1. Every city object contains `MultiPoints` (nodes of LoD0.1), `MultiLineStrings` (edges of LoD0.1) and `MultiSurfaces `(for LoD1+).
2. Every city object contains `MultiLineStrings` (geometry of edges of LoD0.1) and `MultiSurfaces` (for LoD1+). The network topology (nodes and edges) is defined in its own `"+network"` portion of the city model.
3. Every city object has its own `MultiSurfaces` (for LoD1+). Then the actual network (nodes, edges and their geometry) is stored in `"+network"`.
4. Every node is its own city object. Every edge is its own city object. Every surface is its own city object. Then `CityObjectGroups` are used to relate them.

### Fix for `fiona`

*`fiona` has an issue with GDAL 3.0. Better set your `GDAL_DATA` path to fiona's installation (contains GDAL 2.4.4) prior to running this script.*

In [ ]:
import geopandas
import pandas as pd
import osmnx as ox
import networkx as nx

# Load the OSM data

This will load the osm file to a network

In [ ]:
# We don't want edges to be loaded twice
ox.config(all_oneway=True)

network_graph = ox.graph_from_file('../data/Breda/Breda extract.osm', simplify=False)
# We simplify the graph afterwards, so that osmids are not "smashed" together
network_graph = ox.simplify_graph(network_graph, strict=False)
edges = ox.graph_to_gdfs(network_graph, nodes=False, node_geometry=False)

edges = edges.to_crs("EPSG:28992")

len(edges)

In [ ]:
n, e = ox.graph_to_gdfs(network_graph)

## Play with the graph data
Those a little experimentation for me (nothing useful here).

In [ ]:
e.iloc[0]

n.loc[[e.iloc[0]['u'], e.iloc[0]['v']]]

In [ ]:
e.iloc[0]

In [ ]:
ec = ['r' if data['oneway'] else 'b' for u, v, key, data in network_graph.edges(keys=True, data=True)]
ox.plot_graph(network_graph, file_format='svg', save=True, filename='ways', fig_height=15, edge_color=ec)

In [ ]:
# from keys import google_elevation_api_key

## Filter drive and service roads only

Filter roads to only motorways:

In [ ]:
highway_types = [
    "primary",
    "secondary",
    "motorway",
    "trunk",
    "tertiary",
    "unclassified",
    "residential",

    "motorway_link",
    "trunk_link",
    "primary_link",
    "secondary_link",
    "tertiary_link",

    "living_street",
    "service",
    "pedestrian",
    "track",
    "bus_guideway",
    "escape",
    "raceway",
    "road"
]

road_edges = edges[(edges['highway'].isin(highway_types)) & (edges['area'] != 'yes')]

road_edges.to_file('output/road_edges.geojson', driver='GeoJSON')
len(road_edges)

Now `road_edges` has duplicate edges (because twoway roads are modeled as two opposite order linestrings). Let's discard those duplicates (we'll rely on semantics here):

In [ ]:
def lexicographic_uv(f):
    if f['u'] > f['v']:
        return str(f['v']) + "-" + str(f['u'])
    else:
        return str(f['u']) + "-" + str(f['v'])

road_edges = road_edges.copy()
    
road_edges['uv'] = road_edges.apply(lexicographic_uv, axis=1)

# Load the BGT data

In [ ]:
bgt_roads = geopandas.read_file('../data/Breda/bgt_roads.geojson')

len(bgt_roads)

## Compute primary-secondary road polygons

We need to assign secondary road polygons (parking spaces, sidewalks) to the primary roads

### Filter primary roads

Primary types are those that vehicles are supposed to move:

In [ ]:
bgt_road_types = [
    'rijbaan lokale weg',
    'rijbaan regionale weg',
    # 'overweg' # Maybe?
]

bgt_roads =  bgt_roads[bgt_roads['eindRegistratie'].isnull()]

bgt_roads['parent_gml_id'] = bgt_roads['gml_id']

bgt_main_roads = bgt_roads[bgt_roads['function'].isin(bgt_road_types)]

len(bgt_main_roads)

### Identify parent for every secondary road

Find the secondary roads that touch the main ones:

In [ ]:
bgt_secondary_roads = bgt_roads[~bgt_roads['function'].isin(bgt_road_types)]

len(bgt_secondary_roads)

#### Old way
**TODO:** We need to improve the way that main road is assigned (get the one with the highest shared boundary, not just a random one)

In [ ]:
joined = geopandas.sjoin(bgt_secondary_roads, bgt_main_roads, how='left', op='intersects')

joined = joined[['gml_id_left', 'function_left', 'gml_id_right', 'geometry']]
joined.columns = ['gml_id', 'function', 'parent_gml_id', 'geometry']

bgt_roads_parented = geopandas.GeoDataFrame(joined.groupby('gml_id').aggregate('first').reset_index(), crs='EPSG:28992')

len(bgt_roads_parented)

#### New way
Let's overlay the secondary roads with the main roads. The result should be (mostly) linestrings accross their common boundaries:

In [ ]:
bgt_common_boundaries = geopandas.overlay(bgt_secondary_roads, bgt_main_roads, how='intersection', keep_geom_type=False)

len(bgt_common_boundaries)

We need to compute the longest common boundary per `gml_id` to pick the respective parent `gml_id`:

In [ ]:
bgt_common_boundaries['length'] = bgt_common_boundaries['geometry'].length

max_rows = bgt_common_boundaries[['gml_id_1', 'function_1', 'gml_id_2', 'geometry', 'length']].groupby('gml_id_1')['length'].idxmax()

Now, let's join this with the original secondary rows table to bring back their original geometries:

In [ ]:
temp_roads = bgt_common_boundaries.loc[max_rows]
temp_roads = temp_roads[['gml_id_1', 'function_1', 'gml_id_2', 'geometry']]
temp_roads = temp_roads.rename(columns={'gml_id_1': 'gml_id', 'function_1': 'function', 'gml_id_2': 'parent_gml_id'})
temp_roads = temp_roads.set_index('gml_id')

bgt_roads_parented = temp_roads.merge(bgt_secondary_roads.set_index('gml_id'), on='gml_id', suffixes=('', '_right'))
bgt_roads_parented = bgt_roads_parented[['function', 'parent_gml_id', 'geometry_right']].rename(columns={'geometry_right': 'geometry'})
bgt_roads_parented = bgt_roads_parented.reset_index()

bgt_roads_parented = geopandas.GeoDataFrame(bgt_roads_parented, crs='EPSG:28992')

len(bgt_roads_parented)

### Merge primary and secondary roads

Concatenate everything:

In [ ]:
bgt_all_roads = pd.concat([bgt_roads_parented, bgt_main_roads[['gml_id', 'function', 'parent_gml_id', 'geometry']]])

bgt_all_roads.to_file('output/bgt_all_roads.geojson', driver='GeoJSON')

len(bgt_all_roads)

# Intersect roads network with polygon
First, we intersect the road lines with the BGT polygons (to create nodes at the polygon boundaries):

In [ ]:
roads = geopandas.overlay(road_edges, bgt_all_roads, how='intersection')

Export the intersected roads to `GeoJSON`:

In [ ]:
roads.to_file('output/roads.json', driver='GeoJSON')

roads.head()

# Export to CityJSON

Let's export everything to CityJSON.

First, we'll define how the established intersected lines will be translated to `Road` objects:

In [ ]:
def process_ring(line, vertices):
    """Returns the boundaries array of a single linear rings.
    
    Arguments:
    line -- The LineString to be processed (represents the ring)
    vertices -- the global vertices list of the CityJSON
    """
    points = [[x, y, 0] for x, y in list(line.coords)]
    indices = [i + len(vertices) for i in range(len(points))]
    for p in points:
        vertices.append(p)

    return indices

def process_linestring(geom, vertices):
    """Returns the boundaries array of a linear element.
    
    This can process LineString or MultiLineString geometries.
    
    Arguments:
    geom -- the (linear) geometry to be processed
    vertices -- the global vertices list of the CityJSON
    """
    if geom.type == "LineString":
        indices = [process_ring(geom, vertices)]
    else:
        indices = []
        
        for l in geom.geoms:
            indices.append(process_ring(l, vertices))
        
    return indices

def create_geometry(geom, lod, vertices):
    """Returns a CityJSON geometry from a single shapely geometry.
    
    This will also append the 'vertices' list with new vertices.
    
    Arguments:
    geom -- the shapely geometry to be processed
    lod -- the lod of the resulting CityJSON geometry
    vertices -- the global vertices list of the CityJSON
    """
    indices = []
    
    if geom.type == "LineString" or geom.type == "MultiLineString":
        geom_type = "MultiLineString"
        
        indices = process_linestring(geom, vertices)
    elif geom.type == "Polygon":
        geom_type = "MultiSurface"
        
        indices = [process_linestring(geom.boundary, vertices)]
    else:
        raise TypeError(geom.type)
    
    return {
        "type": geom_type,
        "lod": lod,
        "boundaries": indices
    }

def create_geom_with_semantics(map_func, features, lod, vertices, geom_column='geometry'):
    """Returns a CityJSON geometry with semantic elements from multiple features.
    
    Semantic elements are semantic lines (for LineString) or surfaces (for
    MultiSurface).
    
    Arguments:
    map_func -- function that maps an input feature to an output semantic element
    features -- list of features to be processed
    lod -- the lod of the resulting CityJSON geometry
    vertices -- the global vertices list of the CityJSON
    """
    boundaries = []
    semantics = {
        "surfaces": [],
        "values": []
    }
    
    # Get the resulting geometry type from the first feature
    geom = features[0][geom_column]
    if geom.type == "LineString" or geom.type == "MultiLineString":
        geom_type = "MultiLineString"
    elif geom.type == "Polygon":
        geom_type = "MultiSurface"
    else:
        raise TypeError(geom.type)
    
    # Process all features
    i = 0
    for f in features:
        surface = map_func(f)
        
        if geom_type == "MultiLineString":
            indices = process_linestring(f[geom_column], vertices)
            for l in indices:
                boundaries.append(l)
                semantics["surfaces"].append(surface)
                semantics["values"].append(i)
                i = i + 1
        else:
            indices = process_linestring(f[geom_column].boundary, vertices)
        
            boundaries.append(indices)
            semantics["surfaces"].append(surface)
            semantics["values"].append(i)
            i = i + 1
    
    return {
        "type": geom_type,
        "lod": lod,
        "boundaries": boundaries,
        "semantics": semantics
    }

def osm_semantics_map(feature):
    obj = {
        "type": feature['highway'],
        "osm_id": feature['osmid'],
        "oneway": feature['oneway'] if isinstance(feature['oneway'], str) else "no"
    }
    
    if isinstance(feature['name'], str):
        obj["name"] = feature['name']
    
    if isinstance(feature['maxspeed'], str):
        obj["maxspeed"] = feature['maxspeed']
    
    return obj

def bgt_semantics_map(feature):
    return {
        "type": feature['function'],
        "gml_id": feature['gml_id']
    }

def create_cityobject(feature, vertices):
    """Create a CityJSON city object from a single OSM linear features."""
    return {
        "type": "Road",
        "attributes": {
            "osm_id": feature['osmid'],
            "highway": feature['highway'],
        },
        "geometry": [
            create_geometry(feature['geometry'], "0.1", vertices)
        ]
    }

Now, let's run this against all intersected road segments:

In [ ]:
vertices = []
objects = {}

road_groups = roads.groupby('parent_gml_id')

for gml_id, group in road_groups:
    features = [f for i, f in group.iterrows()]
    objects[gml_id] = {
        "type": "Road",
        "geometry": [
            create_geom_with_semantics(osm_semantics_map,
                                       features,
                                       '0.1',
                                       vertices)
        ]
    }

bgt_roads_groups = bgt_all_roads.groupby('parent_gml_id')

for main_gml_id, group in bgt_roads_groups:
    if main_gml_id in objects:
        features = [f for i, f in group.iterrows()]
        objects[main_gml_id]['geometry'].append(
            create_geom_with_semantics(bgt_semantics_map,
                                       features,
                                       '2',
                                       vertices)
        )

Finally, let's export everything as CityJSON:

In [ ]:
import json
import io

output = {
  "type": "CityJSON",
  "version": "1.0",
  "CityObjects": objects,
  "vertices": vertices
}

with open('output/breda.json', 'w') as file:
    json.dump(output, file)

## Fix missing network and polygons
We should create individual road objects for the polygons (and respective network segments) that do not have a `parent_gml_id`.

# Calculate carriageways and lanes
- Compute the carriageways only from OSM attributes.
- Compute road width:
     - We exclude intersections.
- How we create the carriageways for twoway streets and more specifically for intersections.

NOTE: Case of a two-way road where we figure out that the road width doesn't fit two lanes (special case were two ways share the same carriageway).

## Create perpendicular lines

In [ ]:
from shapely.ops import unary_union
from shapely.geometry import Point, LineString

offset = 100

def get_vertex(line, first=True):
    """Returns the first or last vertex"""
    if line.type == "LineString":
        return line.boundary[0 if first else len(line.coords) - 1]
    else:
        return Point(0, 0)
    

def get_perpendicular_line(geom):
    if geom.type == 'LineString':
        left = geom.parallel_offset(offset, 'left')
        right = geom.parallel_offset(offset, 'right')
        a = get_vertex(left, False)
        b = get_vertex(right, True)
        return LineString([a, b])
    else:
        lines = []
        for line in geom.geoms:
            lines.append(get_perpendicular_line(line))
        
        return unary_union(lines)

roads['perp_line'] = roads['geometry'].apply(get_perpendicular_line)

## Calculate width
Let's join the network lines with the polygons:

In [ ]:
common = roads.set_index('parent_gml_id').merge(bgt_all_roads.set_index('gml_id'), on='gml_id', suffixes=('', '_right'))

common.columns

### Old (and slow) way:
We can compute the road width by computing perpendicular lines at one end of the network line and intersecting that with the polygon of the road. This is unreliable and slow.

In [ ]:
def compute_width(polygon, line):
    return polygon.intersection(line).length

def compute_road_width(f):
    return compute_width(f['geometry_right'], f['perp_line'])

# common['road_width'] = common.apply(compute_road_width, axis=1)

# common[['osmid', 'road_width', 'geometry']].to_file('output/road_widths.geojson', driver='GeoJSON')

### New (and fast) way:
Why don't we just assume that the line equals with the length of the side of the road? Then, if we take the polygons perimeter, it's composed by twice the length of the road and twice the width. Easy, isn't it?

In [ ]:
def compute_width_from_perimeter(f):
    return f['geometry_right'].length / 2 - f['geometry'].length

common_dissolved = common.dissolve(by='gml_id')

common_dissolved['width'] = common_dissolved.apply(compute_width_from_perimeter, axis=1)

In [ ]:
common_dissolved.reset_index()[['gml_id', 'width', 'geometry']].to_file('output/width_by_perimeter.geojson', driver='GeoJSON')

# Introduce flyovers

# Validation and evaluation

- Ensure completeless of network (run a simple routing problem)
- Compute decorations like traffic lights
- Potentionally compute more interesting algorithms, e.g. what the best route about de-icing given the width of street (exploiting the semantic surfaces here).